#### *ISEL - DEI - LEIM*
## Aprendizagem Automática [T52D]
### Trabalho Laboratorial 2: Classificação de Críticas de Cinema do IMDb

João Madeira ($48630$), 
Renata Góis ($51038$),
Bruno Pereira ($51811$)

**Docentes responsáveis:** 
- Prof. Gonçalo Xufre Silva

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as p
import re
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
with open("resources/imdbFull.p", "rb") as f:
    D = p.load(f)
print("Keys:", D.keys())

reviews = D['data']
sentiments = D['target']

print(len(reviews), "reviews")

Keys: dict_keys(['data', 'target', 'DESCR'])
50000 reviews


This dataset contains movie reviews along with their associated binary sentiment polarity labels. It is intended to serve as a benchmark for sentiment classification. This document outlines how the dataset was gathered, and how to use the files provided.
For more details see: http://ai.stanford.edu/~amaas/data/sentiment/

GPT answear when I asked what are my options for stemmers

| Method                   | Aggressiveness | Quality | Speed  | Best for                    |
| ------------------------ | -------------- | ------- | ------ | --------------------------- |
| **Porter**               | Medium         | ✔✔      | Fast   | Classic NLP                 |
| **Snowball**             | Medium         | ✔✔✔     | Fast   | Best stemmer for English    |
| **Lancaster**            | High           | ✔       | Fast   | Rare cases; very aggressive |
| **Lemmatizer (spaCy)**   | Low            | ⭐⭐⭐⭐    | Medium | Best accuracy               |
| **Lemmatizer (WordNet)** | Low            | ⭐⭐⭐     | Medium | Simpler lemmatization       |

So I opted for the "Best stemmer for english" since that what we are doing and for what I gathered Lemmatizer $ \not= $ Stemmer

In [3]:
stemmer = SnowballStemmer("english")

def clean_review(string):
    # Remove tags HTML
    string = string.replace('<br />', ' ')  
    # Remove palavras com 20 ou mais caracteres
    string = re.sub(r'\b[a-zA-Z]{20,}\b', ' ', string)
    # Remove palavras com 3 ou mais letras repetidas consecutivamente (e.g., "yaaass", "omgggg")
    string = re.sub(r'\b\w*(.)\1{2,}\w*\b', ' ', string)
    # Filtra apenas letras
    string = re.sub(r'[^a-zA-Z]', ' ', string)
    # Remove espaços consecutivos
    string = re.sub(r'\s+', ' ', string).strip()
    # Normaliza para minúsculas
    string = string.lower()
    # Aplica Stemming
    string = " ".join(stemmer.stem(w) for w in string.split())
    return string

reviews = [clean_review(rev) for rev in reviews]

output = {"data" : reviews, "target" : sentiments}
p.dump(output,open("imdbPreProcessed.p",'wb'))

In [7]:
tfidVector = TfidfVectorizer(min_df=8,                     # Remove palavras que aparecem menos de 8 vezes no dataset
                        max_df=0.9,                        # Remove palavras que aparecem em 90% do dataset
                        ngram_range=(1,2),                 # Utiliza unigramas e bigramas (good, very good, pretty bad)
                        token_pattern=r'\b[a-zA-Z]{3,}\b', # Ignora palavras com menos de 3 letras
                        sublinear_tf=True,                 # Term frequency passa a ter um comportamento logarítmico em vez de linear
                        stop_words='english'               # Remove stopwords em inglês
                        )

In [10]:
pre_processed_data = p.load(open("imdbPreProcessed.p","rb"))
reviews = pre_processed_data['data']
sentiments = pre_processed_data['target']
# tfidVector = tfidVector.fit(reviews)
# tokens = tfidVector.get_feature_names_out()
# X = tfidVector.transform(reviews)
# len(tokens)

#### **1.** Divisão dos dados
---
Originalmente, fizemos o *fit* do TF-IDF em **todos os dados** ($reviews$). No entanto, o vocabulário deve ser aprendido apenas no conjunto de **Treino** e depois aplicado ($transform$) ao conjunto de **Teste**.

In [ ]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    reviews, sentiments, test_size=0.2, random_state=42, stratify=sentiments
)

# Fit do TF-IDF apenas no treino
TfidfVectorizer(min_df=8,                          # Remove palavras que aparecem menos de 8 vezes no dataset
                max_df=0.9,                        # Remove palavras que aparecem em 90% do dataset
                ngram_range=(1,2),                 # Utiliza unigramas e bigramas (good, very good, pretty bad)
                token_pattern=r'\b[a-zA-Z]{3,}\b', # Ignora palavras com menos de 3 letras
                sublinear_tf=True,                 # Term frequency passa a ter um comportamento logarítmico em vez de linear
                stop_words='english')              # Remove stopwords em inglês

X_train = tfidVector.fit_transform(X_train_raw)
X_test = tfidVector.transform(X_test_raw)

print(f"Dimensão do vocabulário: {len(tfidVector.get_feature_names_out())}")

Dimensão do vocabulário: 75754
